In [113]:
import pandas as pd 
import numpy as np 
import configparser
import os
import seaborn as sns
import plotly_express as px

import nltk 

config = configparser.ConfigParser()
config.read("env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'
out_path = f'{output_dir}/{data_prefix}'
OHCO = ['book_id','chap_id','sec_id','para_num', 'sent_num', 'token_num']


os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path)

In [114]:
CORPUS = pd.read_csv(f"{output_dir}\{data_prefix}-CORPUS.csv", sep="|").set_index(OHCO)
LIB = pd.read_csv(f"{output_dir}\{data_prefix}-LIB.csv", sep="|").set_index('book_id')
LIB2 = pd.read_csv(f"{output_dir}\{data_prefix}-LIB2.csv", sep="|").set_index(OHCO[:2])
VOCAB = pd.read_csv(f"{output_dir}\{data_prefix}-VOCAB.csv", sep="|").set_index("term_str")
TFIDF = pd.read_csv(f"{output_dir}\{data_prefix}-TFIDF_chap_L2.csv", sep="|").set_index(OHCO[:2])

<string>:1: SyntaxWarning:

invalid escape sequence '\{'

<string>:2: SyntaxWarning:

invalid escape sequence '\{'

<string>:3: SyntaxWarning:

invalid escape sequence '\{'

<string>:4: SyntaxWarning:

invalid escape sequence '\{'

<string>:5: SyntaxWarning:

invalid escape sequence '\{'

<>:1: SyntaxWarning:

invalid escape sequence '\{'

<>:2: SyntaxWarning:

invalid escape sequence '\{'

<>:3: SyntaxWarning:

invalid escape sequence '\{'

<>:4: SyntaxWarning:

invalid escape sequence '\{'

<>:5: SyntaxWarning:

invalid escape sequence '\{'

<string>:1: SyntaxWarning:

invalid escape sequence '\{'

<string>:2: SyntaxWarning:

invalid escape sequence '\{'

<string>:3: SyntaxWarning:

invalid escape sequence '\{'

<string>:4: SyntaxWarning:

invalid escape sequence '\{'

<string>:5: SyntaxWarning:

invalid escape sequence '\{'

<>:1: SyntaxWarning:

invalid escape sequence '\{'

<>:2: SyntaxWarning:

invalid escape sequence '\{'

<>:3: SyntaxWarning:

invalid escape sequence '\{'

<>:4

In [115]:
TFIDF

abandon  abandoned  abandoning  abandonment  abhimanyu  \
book_id chap_id                                                            
1       1        0.000000   0.000000    0.000000      0.00000   0.006655   
        2        0.000000   0.000000    0.013506      0.00000   0.044282   
        3        0.000000   0.000000    0.000000      0.00000   0.000000   
        4        0.000000   0.000000    0.000000      0.00000   0.000000   
        5        0.000000   0.001717    0.000000      0.00000   0.005015   
...                   ...        ...         ...          ...        ...   
15      2        0.007413   0.000000    0.000000      0.00000   0.009007   
        3        0.000000   0.018916    0.033702      0.00000   0.000000   
16      1        0.000000   0.000000    0.007309      0.00000   0.000000   
17      1        0.048670   0.030372    0.013528      0.04314   0.000000   
18      1        0.000000   0.000000    0.000000      0.00000   0.021942   

                     able  ablutions     abode  abode yama    abodes  ...  \
book_id chap_id                                                       ...   
1       1        0.008051   0.000000  0.009763         0.0  0.000000  ...   
        2        0.000000   0.000000  0.012992         0.0  0.000000  ...   
        3        0.010638   0.014411  0.024079         0.0  0.000000  ...   
        4        0.006363   0.012930  0.012346         0.0  0.000000  ...   
        5        0.005055   0.006163  0.009808         0.0  0.001790  ...   
...                   ...        ...       ...         ...       ...  ...   
15      2        0.010896   0.000000  0.015856         0.0  0.019288  ...   
        3        0.000000   0.022635  0.010806         0.0  0.000000  ...   
16      1        0.004831   0.000000  0.009374         0.0  0.008552  ...   
17      1        0.008943   0.000000  0.000000         0.0  0.000000  ...   
18      1        0.008848   0.000000  0.021458         0.0  0.007831  ...   

                 yudhishthira  yudhishthira just  yudhishthira said  \
book_id chap_id                                                       
1       1            0.016216           0.002479           0.000000   
        2            0.131872           0.010995           0.000000   
        3            0.000000           0.000000           0.000000   
        4            0.000000           0.000000           0.000000   
        5            0.001810           0.000000           0.000000   
...                       ...                ...                ...   
15      2            0.078034           0.006710           0.013565   
        3            0.129635           0.013719           0.027736   
16      1            0.017300           0.000000           0.000000   
17      1            0.224154           0.066080           0.100199   
18      1            0.118807           0.021793           0.005508   

                 yudhishthira son  yudhishthiras  yudhisthira      yuga  \
book_id chap_id                                                           
1       1                0.000000       0.000000          0.0  0.006320   
        2                0.000000       0.044282          0.0  0.000000   
        3                0.000000       0.000000          0.0  0.000000   
        4                0.000000       0.000000          0.0  0.000000   
        5                0.001345       0.000000          0.0  0.014288   
...                           ...            ...          ...       ...   
15      2                0.000000       0.000000          0.0  0.000000   
        3                0.000000       0.000000          0.0  0.000000   
16      1                0.000000       0.000000          0.0  0.000000   
17      1                0.000000       0.000000          0.0  0.000000   
18      1                0.000000       0.000000          0.0  0.000000   

                    yugas  yuyudhana   yuyutsu  
book_id chap_id                                 
1       1        0.004751   0

In [116]:
from sklearn.decomposition import PCA
pca_engine = PCA(n_components=10)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF), index=TFIDF.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM


PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1       -0.003300 -0.467583 -0.188407 -0.142955  0.256762  0.148995   
        2        0.055169 -0.335624  0.018964 -0.132982  0.106272 -0.206290   
        3       -0.204875 -0.152495 -0.233734  0.091223  0.065066  0.175992   
        4       -0.237738 -0.151144 -0.340371  0.143830  0.020320  0.053529   
        5       -0.201264 -0.070424 -0.169411  0.065874 -0.094527  0.097034   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.131765  0.113481  0.042277 -0.206329 -0.031878 -0.045305   
        3       -0.104443 -0.055151  0.177017 -0.263387  0.076764  0.209946   
16      1        0.095710 -0.147082 -0.052212 -0.109657 -0.185852 -0.115741   
17      1       -0.139510  0.009462  0.065420 -0.110708 -0.067713  0.067395   
18      1       -0.100783  0.044538 -0.005871 -0.274412 -0.131658  0.006344   

                      PC6       PC7       PC8       PC9  
book_id chap_id                                          
1       1       -0.281964 -0.307718  0.232038 -0.022482  
        2        0.032253 -0.186334  0.254485 -0.098505  
        3       -0.124982 -0.168900  0.075170  0.182410  
        4        0.180064 -0.083679  0.386193  0.137151  
        5        0.092406 -0.049524  0.337898  0.109735  
...                   ...       ...       ...       ...  
15      2        0.213546 -0.181161 -0.013159  0.026482  
        3        0.196977 -0.196364 -0.034165 -0.050209  
16      1        0.124085 -0.296272 -0.268130  0.132545  
17      1        0.157373 -0.210723 -0.223558 -0.034564  
18      1        0.116007 -0.102762 -0.090246 -0.065068  

[99 rows x 10 columns]

In [117]:
DCM = DCM.join(LIB2[['book_name','chap_name']], on=OHCO[:2])
DCM

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1       -0.003300 -0.467583 -0.188407 -0.142955  0.256762  0.148995   
        2        0.055169 -0.335624  0.018964 -0.132982  0.106272 -0.206290   
        3       -0.204875 -0.152495 -0.233734  0.091223  0.065066  0.175992   
        4       -0.237738 -0.151144 -0.340371  0.143830  0.020320  0.053529   
        5       -0.201264 -0.070424 -0.169411  0.065874 -0.094527  0.097034   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.131765  0.113481  0.042277 -0.206329 -0.031878 -0.045305   
        3       -0.104443 -0.055151  0.177017 -0.263387  0.076764  0.209946   
16      1        0.095710 -0.147082 -0.052212 -0.109657 -0.185852 -0.115741   
17      1       -0.139510  0.009462  0.065420 -0.110708 -0.067713  0.067395   
18      1       -0.100783  0.044538 -0.005871 -0.274412 -0.131658  0.006344   

                      PC6       PC7       PC8       PC9  \
book_id chap_id                                           
1       1       -0.281964 -0.307718  0.232038 -0.022482   
        2        0.032253 -0.186334  0.254485 -0.098505   
        3       -0.124982 -0.168900  0.075170  0.182410   
        4        0.180064 -0.083679  0.386193  0.137151   
        5        0.092406 -0.049524  0.337898  0.109735   
...                   ...       ...       ...       ...   
15      2        0.213546 -0.181161 -0.013159  0.026482   
        3        0.196977 -0.196364 -0.034165 -0.050209   
16      1        0.124085 -0.296272 -0.268130  0.132545   
17      1        0.157373 -0.210723 -0.223558 -0.034564   
18      1        0.116007 -0.102762 -0.090246 -0.065068   

                             book_name              chap_name  
book_id chap_id                                                
1       1                    Adi Parva     Anukramanika Parva  
        2                    Adi Parva         Sangraha Parva  
        3                    Adi Parva          Paushya Parva  
        4                    Adi Parva          Pauloma Parva  
        5                    Adi Parva           Astika Parva  
...                                ...                    ...  
15      2           Asramavasika Parva     Putradarsana Parva  
        3           Asramavasika Parva     Naradagamana Parva  
16      1                Mausala Parva          Mausala Parva  
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva  
18      1         Svargarohanika Parva   Svargarohanika Parva  

[99 rows x 12 columns]

In [118]:
def vis_pcs(M, a, b, hover_name='chap_name', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", 
                      color = "book_name" ,
                    hover_name=hover_name, 
                    symbol=symbol, size=size,
                    marginal_x='box', height=800, width = 1200)

In [119]:
vis_pcs(DCM, 0, 1)

In [120]:
vis_pcs(DCM, 1, 2)

In [121]:
vis_pcs(DCM, 2, 3)

In [122]:
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.index = TFIDF.columns
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index.name = 'term_str'
LOADINGS2 = LOADINGS.join(VOCAB, on="term_str")
LOADINGS2

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,stop,stem_porter,stem_snowball,stem_lancaster,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos
term_str,,,,,,,,,,,,,,,,,,,,,
abandon,-0.001039,0.001530,1.015186e-03,0.000724,0.001762,-0.001781,0.002390,0.000144,-0.000474,0.000571,...,0.0,abandon,abandon,abandon,VB,VB,5.0,"{'IN', 'NN', 'VB', 'JJ', 'RB'}",8.0,"{'VBZ', 'IN', 'VB', 'NNP', 'JJ', 'RB', 'VBD', ..."
abandoned,0.000493,0.000774,5.497207e-04,-0.000923,-0.000202,0.000338,-0.000128,0.000048,-0.000378,0.000420,...,0.0,abandon,abandon,abandon,VBN,VB,1.0,{'VB'},2.0,"{'VBD', 'VBN'}"
abandoning,0.000728,0.001721,-3.737182e-04,-0.001065,0.001220,-0.000924,0.001849,0.000606,-0.000008,0.000551,...,0.0,abandon,abandon,abandon,VBG,VB,3.0,"{'JJ', 'VB', 'NN'}",4.0,"{'JJ', 'NN', 'VBG', 'NNP'}"
abandonment,-0.001590,0.000201,-2.348353e-03,-0.001912,0.001559,-0.002049,-0.000412,0.001934,-0.001244,0.000679,...,0.0,abandon,abandon,abandon,NN,NN,2.0,"{'VB', 'NN'}",3.0,"{'NNP', 'VB', 'NN'}"
abhimanyu,0.007394,0.000379,-1.408907e-03,-0.001985,-0.004366,-0.003788,-0.001723,-0.004326,0.001460,0.003192,...,0.0,abhimanyu,abhimanyu,abhimanyu,NNP,NN,3.0,"{'RB', 'VB', 'NN'}",4.0,"{'VBP', 'RB', 'VB', 'NNP'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yudhisthira,-0.000886,-0.002202,5.548282e-03,-0.002082,-0.002852,0.001105,-0.002093,0.002391,0.000295,0.000479,...,0.0,yudhisthira,yudhisthira,yudhisthir,NNP,NN,2.0,"{'RB', 'NN'}",3.0,"{'RB', 'NN', 'NNP'}"
yuga,0.002103,0.001693,-1.872818e-03,-0.000821,-0.001134,-0.000018,-0.000162,0.000848,0.001095,-0.000326,...,0.0,yuga,yuga,yug,NNP,NN,1.0,{'NN'},2.0,"{'NN', 'NNP'}"
yugas,-0.000749,-0.000097,-5.386320e-07,-0.000713,-0.000377,-0.000599,-0.000510,-0.000195,0.000836,-0.000667,...,0.0,yuga,yuga,yuga,NNP,NN,2.0,"{'CD', 'NN'}",4.0,"{'NNS', 'NN', 'CD', 'NNP'}"


In [123]:
LOADINGS

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
term_str,,,,,,,,,,
abandon,-0.001039,0.001530,1.015186e-03,0.000724,0.001762,-0.001781,0.002390,0.000144,-0.000474,0.000571
abandoned,0.000493,0.000774,5.497207e-04,-0.000923,-0.000202,0.000338,-0.000128,0.000048,-0.000378,0.000420
abandoning,0.000728,0.001721,-3.737182e-04,-0.001065,0.001220,-0.000924,0.001849,0.000606,-0.000008,0.000551
abandonment,-0.001590,0.000201,-2.348353e-03,-0.001912,0.001559,-0.002049,-0.000412,0.001934,-0.001244,0.000679
abhimanyu,0.007394,0.000379,-1.408907e-03,-0.001985,-0.004366,-0.003788,-0.001723,-0.004326,0.001460,0.003192
...,...,...,...,...,...,...,...,...,...,...
yudhisthira,-0.000886,-0.002202,5.548282e-03,-0.002082,-0.002852,0.001105,-0.002093,0.002391,0.000295,0.000479
yuga,0.002103,0.001693,-1.872818e-03,-0.000821,-0.001134,-0.000018,-0.000162,0.000848,0.001095,-0.000326
yugas,-0.000749,-0.000097,-5.386320e-07,-0.000713,-0.000377,-0.000599,-0.000510,-0.000195,0.000836,-0.000667


In [124]:
top_terms_sk= {}
data = []
for i in range(10):
    for j in [0, 1]:
        data.append((i, j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))

In [125]:
comp_strs = pd.DataFrame(data)
comp_strs.columns =  ['pc', 'pole', 'gloss']
comp_strs = comp_strs.set_index(['pc', 'pole'])

In [126]:
comp_strs.unstack()

gloss  \
pole                                                  0   
pc                                                        
0     battle shafts arrows son drona steeds pierced ...   
1     thou thy thee son king said battle words art t...   
2     king pandavas kings krishna yudhishthira sons ...   
3     thou rakshasa thy unto thee bhima vaisampayana...   
4     vidura pandavas dhritarashtra duryodhana sanja...   
5     king thou nala island virata damayanti lord mo...   
6     rakshasa bhima great king hidimva brahmana gan...   
7     rakshasa jarasandha like king bhima men mighty...   
8     sauti called bhrigu astika sanjaya parva conti...   
9     jarasandha bhishma krishna utanka rama daughte...   

                                                         
pole                                                  1  
pc                                                       
0     thou thee said brahmana brahmanas unto shall v...  
1     rakshasa success heard hope success sanjaya ho...  
2     rakshasa brahman utanka brahmana shafts sauti ...  
3     king great called sanjaya brahman deities know...  
4     unto great arjuna god celestials krishna brahm...  
5     pandavas arjuna vidura son sons krishna unto h...  
6     hath krishna bhishma success heard hope succes...  
7     success heard vaishampayana sanjaya hope hope ...  
8     yudhishthira vaishampayana vrishnis like grief...  
9     yudhishthira mighty celestials matali having a...

In [127]:
comp_strs

gloss
pc pole                                                   
0  0     battle shafts arrows son drona steeds pierced ...
   1     thou thee said brahmana brahmanas unto shall v...
1  0     thou thy thee son king said battle words art t...
   1     rakshasa success heard hope success sanjaya ho...
2  0     king pandavas kings krishna yudhishthira sons ...
   1     rakshasa brahman utanka brahmana shafts sauti ...
3  0     thou rakshasa thy unto thee bhima vaisampayana...
   1     king great called sanjaya brahman deities know...
4  0     vidura pandavas dhritarashtra duryodhana sanja...
   1     unto great arjuna god celestials krishna brahm...
5  0     king thou nala island virata damayanti lord mo...
   1     pandavas arjuna vidura son sons krishna unto h...
6  0     rakshasa bhima great king hidimva brahmana gan...
   1     hath krishna bhishma success heard hope succes...
7  0     rakshasa jarasandha like king bhima men mighty...
   1     success heard vaishampayana sanjaya hope hope ...
8  0     sauti called bhrigu astika sanjaya parva conti...
   1     yudhishthira vaishampayana vrishnis like grief...
9  0     jarasandha bhishma krishna utanka rama daughte...
   1     yudhishthira mighty celestials matali having a...

In [128]:
def vis_loadings(LOADINGS, a=0, b=1, hover_name='term_str'):
    # LOADINGS = LOADINGS[LOADINGS['max_pos_group'].apply(lambda x: x in ['NN','VB','JJ'])]
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                       color='max_pos_group', 
                      marginal_x='box', height=800)

In [129]:
vis_loadings(LOADINGS2, 1, 2)

In [130]:
TFIDF

term_str          abandon  abandoned  abandoning  abandonment  abhimanyu  \
book_id chap_id                                                            
1       1        0.000000   0.000000    0.000000      0.00000   0.006655   
        2        0.000000   0.000000    0.013506      0.00000   0.044282   
        3        0.000000   0.000000    0.000000      0.00000   0.000000   
        4        0.000000   0.000000    0.000000      0.00000   0.000000   
        5        0.000000   0.001717    0.000000      0.00000   0.005015   
...                   ...        ...         ...          ...        ...   
15      2        0.007413   0.000000    0.000000      0.00000   0.009007   
        3        0.000000   0.018916    0.033702      0.00000   0.000000   
16      1        0.000000   0.000000    0.007309      0.00000   0.000000   
17      1        0.048670   0.030372    0.013528      0.04314   0.000000   
18      1        0.000000   0.000000    0.000000      0.00000   0.021942   

term_str             able  ablutions     abode  abode yama    abodes  ...  \
book_id chap_id                                                       ...   
1       1        0.008051   0.000000  0.009763         0.0  0.000000  ...   
        2        0.000000   0.000000  0.012992         0.0  0.000000  ...   
        3        0.010638   0.014411  0.024079         0.0  0.000000  ...   
        4        0.006363   0.012930  0.012346         0.0  0.000000  ...   
        5        0.005055   0.006163  0.009808         0.0  0.001790  ...   
...                   ...        ...       ...         ...       ...  ...   
15      2        0.010896   0.000000  0.015856         0.0  0.019288  ...   
        3        0.000000   0.022635  0.010806         0.0  0.000000  ...   
16      1        0.004831   0.000000  0.009374         0.0  0.008552  ...   
17      1        0.008943   0.000000  0.000000         0.0  0.000000  ...   
18      1        0.008848   0.000000  0.021458         0.0  0.007831  ...   

term_str         yudhishthira  yudhishthira just  yudhishthira said  \
book_id chap_id                                                       
1       1            0.016216           0.002479           0.000000   
        2            0.131872           0.010995           0.000000   
        3            0.000000           0.000000           0.000000   
        4            0.000000           0.000000           0.000000   
        5            0.001810           0.000000           0.000000   
...                       ...                ...                ...   
15      2            0.078034           0.006710           0.013565   
        3            0.129635           0.013719           0.027736   
16      1            0.017300           0.000000           0.000000   
17      1            0.224154           0.066080           0.100199   
18      1            0.118807           0.021793           0.005508   

term_str         yudhishthira son  yudhishthiras  yudhisthira      yuga  \
book_id chap_id                                                           
1       1                0.000000       0.000000          0.0  0.006320   
        2                0.000000       0.044282          0.0  0.000000   
        3                0.000000       0.000000          0.0  0.000000   
        4                0.000000       0.000000          0.0  0.000000   
        5                0.001345       0.000000          0.0  0.014288   
...                           ...            ...          ...       ...   
15      2                0.000000       0.000000          0.0  0.000000   
        3                0.000000       0.000000          0.0  0.000000   
16      1                0.000000       0.000000          0.0  0.000000   
17      1                0.000000       0.000000          0.0  0.000000   
18      1                0.000000       0.000000          0.0  0.000000   

term_str            yugas  yuyudhana   yuyutsu  
book_id chap_id                                 
1       1  